In [1]:
from pyraf import iraf, irafpar
from pyraf.irafpar import IrafParList
import os
import math
iraf.stsdas(_doprint=0)
iraf.analysis(_doprint=0)
iraf.isophote(_doprint=0)



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |
      |                   STSDAS Version 3.18.3                    |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+


In [ ]:
def init():
    
    iraf.flpr()
    iraf.noao(_doprint=0)
    iraf.digiphot(_doprint=0)
    iraf.apphot(_doprint=0)
    iraf.stsdas(_doprint=0)
    iraf.analysis(_doprint=0)
    iraf.isophote(_doprint=0)
    iraf.datapars.unlearn() 
    iraf.fitskyp.unlearn()
    iraf.fitsky.unlearn()
    iraf.tprint.unlearn()
    iraf.imgets.unlearn()
    iraf.mskregions.unlearn()
    iraf.fixpix.unlearn()
    iraf.controlpar.unlearn()
    iraf.geompar.unlearn()
    iraf.ellipse.unlearn()

    sma = 0.0
    intens = 0.0
    tflux = 0.0
    npix = 0.0
    ellip = 0.0
    pa = 0.0
    r_ba = 0.0
    magzp = 0.0
    msky, stdev = 0.0,0.0
    newX0, newY0 = 0., 0.
    photflam = 0.0
    st_mag, ab_mag = 0., 0.
    


In [2]:
def runEllipse(x0, y0, magzp):
    print 'Running ellipse first stage...'
    
    iraf.unlearn(iraf.geompar)
    iraf.unlearn(iraf.controlpar)
    iraf.unlearn(iraf.magpar)
    iraf.unlearn(iraf.ellipse)
    iraf.flprcache()
    iraf.ellipse.x0=x0
    iraf.ellipse.y0=y0
    iraf.ellipse.step=0.1
    iraf.ellipse.linear='no' 
    #iraf.ellipse.wander=3
    iraf.ellipse.maxgerr=1.       #fix maxgerr=1 for all bands    
    iraf.ellipse.mag0=magzp
    iraf.ellipse.input=input_dir+target_name+'.fits'
    iraf.ellipse.output=input_dir+target_name+'_isofit.tab'  
    iraf.ellipse.region = 'no'
    iraf.ellipse.interactive = 'no'
    iraf.ellipse.verbose = 'no'
    iraf.ellipse()   


In [ ]:
# running isophotal aperture photometry only for the 3 sources flagged in WISE archive as 'extended'

targets = ['0258+35','1037+30','1221-423']
bands = ['w1', 'w2', 'w3', 'w4']


In [ ]:
for i in range(len(bands)):
    
    band = bands[i]
    input_dir = '/Users/orion/phd_research/CSS2/ap_phot_wise/'+band+'/'
    uparm_dir = '/Users/orion/phd_research/CSS2/ap_phot_wise/'+band+'/uparm/'
    mask_dir = '/home/chetna/Documents/ap_phot_wise/'+band+'/'
    
    for j in range(len(targets)):
        
        coord_file = '/Users/orion/phd_research/CSS2/css_ap_phot/coord/'+targets[j]+'.txt'
        
        target_name = band+'_'+targets[j]
        print target_name

        init()
        
        #Converting RA and DEC in wcs to pixels
        iraf.wcsctran.setParam('input', coord_file)
        iraf.wcsctran.setParam('image', input_dir+target_name+'.fits')
        iraf.wcsctran.setParam('inwcs', 'world')
        iraf.wcsctran.setParam('outwcs', 'logical')
        iraf.wcsctran.setParam('columns', '1 2')
        iraf.wcsctran.setParam('units', 'd')
        xy = iraf.wcsctran(Stdout=1, mode='h')
        print xy
        xy = xy[3].split()  #Extracting only the x and y pixel position from the output string of wcsctran
        x0 = round(float(xy[0]), 1)
        y0 = round(float(xy[1]),1)
        print 'x0 = ',x0
        print 'y0 = ',y0

        #Creating mask regions using the .reg files of the respective .fits using the mskregions task
        iraf.mskregions.setParam('regions', input_dir+target_name+'_mask.reg')
        iraf.mskregions.setParam('masks', input_dir+target_name+'_mask.fits.pl')
        iraf.mskregions.setParam('refimage', input_dir+target_name+'.fits')
        iraf.mskregions.setParam('append', 'no')
        iraf.mskregions.setParam('verbose', 'no')
        iraf.mskregions.saveParList(filename=uparm_dir+'mskregions.par')
        iraf.mskregions(mode='l')
        print "Masking done"

        #Read photometric zero-point from image header
        iraf.images.imutil.imgets(input_dir+target_name+'.fits', 'magzp')
        magzp = iraf.images.imutil.imgets.value
        #print "magzp = ",magzp

        #Ellipse first run
        runEllipse(x0, y0, magzp) 

        iraf.tdump.setParam('table',input_dir+target_name+'_isofit.tab')
        #iraf.tdump.setParam('table',output_dir+target_name+'_companion_isofit.tab')
        iraf.tdump.setParam('cdfile','null')
        iraf.tdump.setParam('pfile','null')
        iraf.tdump.setParam('datafil', input_dir+target_name+'_isofit.dat')
        iraf.tdump.setParam('columns', 'SMA, intens, int_err, rms, ellip, ellip_err, pa, pa_err, grad, grad_err, rsma, mag, mag_lerr, tflux_e, npix_e, X0, Y0, ndata ')
        #selecting 18 columns from .tab file - and only these are copied to the .dat file 
        iraf.tdump.setParam('pwidth', 300)
        iraf.tdump.saveParList(uparm_dir+'tdump.par')
        iraf.tdump(mode='h')


        for line in reversed(open(input_dir+target_name+'_isofit.dat').readlines()):
            print line
            line = line.strip().split()
            if line[8].strip() !='INDEF' and line[9].strip()!='INDEF':    
                sma = float(line[0].strip())
                break
        for line in reversed(open(input_dir+target_name+'_isofit.dat').readlines()):
            line = line.strip().split()
            if float(line[1].strip()) >= (msky+1.*stdev):    #line[1] is intensity
                maxsma = float(line[0].strip())
            print 'maxsma', maxsma
            break

    

        #Setting datapars for fitsky
        iraf.datapars.setParam('scale', '1.375')            # pixel scale of image (ds9 region stats)
        iraf.datapars.setParam('fwhmpsf', '6')              # FWHM psf in W1-band = 6 arcsec
        iraf.datapars.saveParList(filename = uparm_dir+'fitsky_data.par')

        #setting fitskypars for fitsky
        iraf.fitskyp.setParam('salgori', 'mode')
        iraf.fitskyp.setParam('annulus', sma*1.375+50.*1.375)   #annulus in scale units, so multiplied by "scale" above
        iraf.fitskyp.setParam('dannulus', 20*1.375)
        iraf.fitskyp.saveParList(filename = uparm_dir+'fitsky_fitsky.par')

        #Setting fitsky
        iraf.fitsky.setParam('image', input_dir+target_name+'.fits')
        iraf.fitsky.setParam('coords', coord_file)
        iraf.fitsky.setParam('output',input_dir+target_name+'.sky')
        iraf.fitsky.setParam('datapars', uparm_dir+'fitsky_data.par')
        iraf.fitsky.setParam('fitskypars', uparm_dir+'fitsky_fitsky.par')
        iraf.fitsky.setParam('interactive', 'no')
        iraf.fitsky.setParam('verbose', 'no')
        iraf.fitsky.setParam('verify', 'no')

        print iraf.fitsky.getParam("fitsky.image")
        iraf.fitsky(mode='h', Stdout=1)


        sky_values = []
        #Reading the output of fitsky task
        f = open(input_dir+target_name+'.sky', 'r')
        for line in f:
            if '#' not in line:
                temp = line.strip().split()
                sky_values.append(temp)
        f.close()        
        msky = float(sky_values[1][0])
        stdev = float(sky_values[1][1])
        print "msky = ", msky
        print "stdev = ", stdev


        #   -------------------Fitsky gives error for 1221-423, use manually entered msky & stdev
        #selecting band for the image stats in 1221-423
        if band_1221 == 'w1':
            msky = 2.744
            stdev = 0.123
        elif band_1221 =='w2':
            msky = 7.236
            stdev = 0.063
        elif band_1221 =='w3':
            msky = 421.180
            stdev = 0.278
        else:
            msky = 126.238
            stdev = 0.028

        print 'manually entered stats:', 'msky=', msky, 'stdev=', stdev
        #                        --------------------------------------------


        #Using msky+1sigma as the cut-off for the max sma to consider for ellipse isophote 
        #and also the one that does not have INDEF in the ellip_err and pa_err and grad and 
        # grad_err column of output table.

        #columns='SMA, intens, int_err, rms, ellip, ellip_err, 
        #pa, pa_err, grad, grad_err,rsma, mag, mag_lerr,
        #tflux_e, npix_e, X0, Y0, ndata'
  
        
        for line in reversed(open(input_dir+target_name+'_isofit.dat').readlines()):
            line = line.strip()
            line = line.split()
            if float(line[1].strip()) >= (msky+1.*stdev) and line[9].strip()!='INDEF':     #line[1] is intensity
                newX0 = float(line[15])      #column 15,16 - we designated these columns to x0, y0 in "tdump"
                newY0 = float(line[16])
                sma = float(line[0].strip())
                intens = float(line[1].strip())
                tflux = float(line[13].strip())
                npix = float(line[14].strip())
                ellip = float(line[4].strip())
                pa = float(line[6].strip())
                r_ba = 1 - ellip
                print 'new sma = ', sma
                break

        print "1sigma aperture params:", target_name, newX0, newY0, ellip, pa 

        #Subtracting sky background    
        tflux_bgsub = tflux - npix*msky
        mag = round((-2.5*math.log10(tflux_bgsub) + float(magzp)), 6)
        print "Target values:"
        print "bkgd_sub_flux= ", tflux_bgsub, "total_valid_pixels= ", npix, "mag= ", mag


        #Poisson noise (shot noise)
        n_poisson = math.sqrt(tflux)
        #error in sky
        n_sky = math.sqrt(stdev*npix)
        n_tot = math.sqrt(n_poisson**2 + n_sky**2)    #Adding noise in quadrature
        n_tot_mag = 2.5*math.log10(1.+n_tot/tflux)


        iraf.display.setParam('bpdispl', 'overlay')
        iraf.display.setParam('overlay',  input_dir+target_name+'.fits.pl')

        #output region file with ellipse and text
        region_file = open(input_dir+target_name+'_isophotes.reg','w')
        region_file.write("# Region file format: DS9 version 6.2\n")
        region_file.write('#global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 image\n')
        region_file.write('ellipse(' + str(newX0) + ',' + str(newY0)+ ',' + str(sma) + ','+str(round(sma*r_ba,2)) + ','+ str(90.+pa) +  ')' + '\n')
        region_file.close()

        

w1_0258+35

x0 =  221.0

y0 =  218.0

Masking done

Running ellipse first stage...

81.40277      4.417534    0.01609845     0.3334362     0.2830037   0.007373483      59.09297     0.8761135   -0.03965309         INDEF      3.003722      18.88705   0.003963429      128845.8       14919      234.2318      209.0878         429

74.00252      4.678188      0.020136     0.3976541     0.2830037    0.01785968      59.09297      2.122077   -0.03034485   0.001836808      2.932997      18.82481   0.004682852      116767.6       12349      227.8534      212.9063         390

maxsma 81.40277

/home/chetna/Documents/ap_phot_wise/w1/w1_0258+35.fits

msky =  3.706105

stdev =  0.0393027

new sma =  74.00252

1sigma aperture params: w1_0258+35 227.8534 212.9063 0.2830037 59.09297

Target values:

bkgd_sub_flux=  71000.909355 total_valid_pixels=  12349.0 mag=  8.37184



w1_1037+30

x0 =  218.8

y0 =  218.6

Masking done

Running ellipse first stage...

19.48717      3.272496   0.005788291    0.05902922     0.2605486    0.01566129     -105.6531      1.983415   -0.03806074         INDEF      2.101054       19.2128   0.001921357       4447.42         881      220.6408      218.2815         104

17.71561      3.243932   0.005553793     0.0566378      0.115327    0.05476523     -89.60024      14.37307   -0.01458011   0.004681075      2.051583      19.22232   0.001860416      4408.399         869      219.1192      219.1861         104

maxsma 19.48717

/home/chetna/Documents/ap_phot_wise/w1/w1_1037+30.fits

msky =  3.163443

stdev =  0.01121001

new sma =  17.71561

1sigma aperture params: w1_1037+30 219.1192 219.1861 0.115327 -89.60024

Target values:

bkgd_sub_flux=  1659.367033 total_valid_pixels=  869.0 mag=  12.450144



w1_1221-423

x0 =  219.3

y0 =  223.8

Masking done

Running ellipse first stage...

21.43589      2.997542     0.0191349     0.2069756      0.234606    0.04866391      37.74832      6.759206    -0.0321315         INDEF      2.151718      19.30809   0.006953616       5652.02        1113      220.0614      222.7394         117

19.48717      3.075811    0.01262832     0.1300165      0.234606    0.03040252      37.74832      4.208109   -0.04016437    0.01176486      2.101054       19.2801   0.004466095      5041.396         911      220.0614      222.7394         106

maxsma 21.43589

manually entered stats: msky= 2.744 stdev= 0.123

new sma =  19.48717

1sigma aperture params: w1_1221-423 220.0614 222.7394 0.234606 37.74832

Target values:

bkgd_sub_flux=  2541.612 total_valid_pixels=  911.0 mag=  11.987227



w2_0258+35

x0 =  221.0

y0 =  218.0

Masking done

Running ellipse first stage...

89.54305      9.190417   0.007114586      0.153912     0.2933479         INDEF      64.33739         INDEF  -0.005194466         INDEF      3.076153      17.09166   8.412761E-4      189573.1       17805       231.491      209.2299         468

81.40277      9.243477   0.007109561     0.1465673     0.2933479    0.02338577      64.33739      2.695354  -0.006493083   0.001515184      3.003722      17.08541   8.359493E-4      161036.3       14705       231.491      209.2299         425

maxsma 89.54305

/home/chetna/Documents/ap_phot_wise/w2/w2_0258+35.fits

msky =  9.039528

stdev =  0.01887645

new sma =  81.40277

1sigma aperture params: w2_0258+35 231.491 209.2299 0.2933479 64.33739

Target values:

bkgd_sub_flux=  28110.04076 total_valid_pixels=  14705.0 mag=  8.377846



w2_1037+30

x0 =  218.8

y0 =  218.6

Masking done

Running ellipse first stage...

21.43589      9.078351   0.004323262     0.0445107     0.3564185    0.02527659      -74.2551      2.498897   -0.01347925         INDEF      2.151718      17.10498   5.173345E-4      9142.657         927      217.0589      219.1438         106

19.48717      9.111185   0.002888811    0.02845148     0.3564185    0.01617635      -74.2551      1.609794   -0.01684906   0.002668216      2.101054      17.10106   3.443388E-4      7724.219         771      217.0589      219.1438          97

maxsma 21.43589

/home/chetna/Documents/ap_phot_wise/w2/w2_1037+30.fits

msky =  9.080258

stdev =  0.01291013

new sma =  19.48717

1sigma aperture params: w2_1037+30 217.0589 219.1438 0.3564185 -74.2551

Target values:

bkgd_sub_flux=  723.340082 total_valid_pixels=  771.0 mag=  12.351644



w2_1221-423

x0 =  219.3

y0 =  223.8

Masking done

Running ellipse first stage...

23.57948      7.274249    0.01319907     0.1556147     0.1075632         INDEF      -95.6955         INDEF  -0.007444002         INDEF      2.203604      17.34553   0.001971913      12746.07        1557      219.5793      222.4127         139

 21.43589      7.294195    0.01387934     0.1564122     0.1075632         INDEF      -95.6955         INDEF  -0.009305002   0.008935187      2.151718      17.34256   0.002068514      10835.51        1295      219.5793      222.4127         127

maxsma 23.57948

manually entered stats: msky= 7.236 stdev= 0.063

new sma =  19.48717

1sigma aperture params: w2_1221-423 219.5793 222.4127 0.1075632 -95.6955

Target values:

bkgd_sub_flux=  1446.746 total_valid_pixels=  1069.0 mag=  11.599019




w3_0258+35

x0 =  221.0

y0 =  218.0

Masking done

Running ellipse first stage...

144.21      650.5917    0.02395749     0.5764753     0.5528916         INDEF     -86.42173         INDEF  -0.008644553         INDEF      3.465364      10.96673   4.046469E-5     19024914.       29193      152.6953      216.4992         579

131.1      650.7211      0.028972     0.6650952     0.5528916         INDEF     -86.42173         INDEF   -0.01080569   0.003419335      3.383769      10.96651   4.852218E-5     15735531.       24137      152.6953      216.4992         527

maxsma 144.21

/home/chetna/Documents/ap_phot_wise/w3/w3_0258+35.fits

msky =  650.9052

stdev =  0.5388246

new sma =  55.59918

1sigma aperture params: w3_0258+35 211.6827 221.8656 0.1952937 56.88509

Target values:

bkgd_sub_flux=  25339.1036 total_valid_pixels=  7807.0 mag=  6.990522



w3_1037+30

x0 =  218.8

y0 =  218.6

Masking done

Running ellipse first stage...

25.93743       578.043    0.02461006      0.281675     0.3330231    0.07545193      52.10416      7.861163   -0.02234381         INDEF      2.256741       11.0951   4.633085E-5       813664.        1405      224.6039      216.2425         131

23.57948      578.1089    0.03165304     0.3452939     0.3330231    0.08722392      52.10416      9.080607   -0.02792977    0.01700399      2.203604      11.09498   5.990906E-5      672614.4        1161      224.6039      216.2425         119

maxsma 25.93743

/home/chetna/Documents/ap_phot_wise/w3/w3_1037+30.fits

msky =  578.5399

stdev =  0.069539

new sma =  13.31

1sigma aperture params: w3_1037+30 219.613 220.0489 0.07414997 81.74577

Target values:

bkgd_sub_flux=  1058.3919 total_valid_pixels=  519.0 mag=  10.438384



w3_1221-423

x0 =  219.3

y0 =  223.8

Masking done

Running ellipse first stage...

45.94973      421.1465    0.02494345     0.3456267     0.5236989         INDEF      42.19756         INDEF    -0.0178492         INDEF      2.603579      11.43892   6.417085E-5      1334451.        3153      227.1007      210.4615         192

 41.77248      421.2397    0.02555951     0.3371531     0.5236989         INDEF      42.19756         INDEF    -0.0223115   0.008549563      2.542275      11.43868   6.588515E-5      1106169.        2611      227.1007      210.4615         174

maxsma 45.94973

manually entered stats: msky= 421.18 stdev= 0.278

new sma =  19.48717

1sigma aperture params: w3_1221-423 218.8448 220.9579 0.1639296 42.64727

Target values:

bkgd_sub_flux=  6153.56 total_valid_pixels=  993.0 mag=  8.527184


    
w4_0258+35

x0 =  221.0

y0 =  218.0

Masking done

Running ellipse first stage...

89.54305      201.7642   0.002637889    0.05620622     0.3293196         INDEF      65.49702         INDEF   -0.00219971         INDEF      3.076153       7.23789   1.396802E-5      3410974.       16899      194.2824      218.9423         454

81.40277      201.7989   0.001833499    0.03726111     0.3293196     0.0100439      65.49702      1.057016  -0.004296309   4.956382E-4      3.003722      7.237703   1.007699E-5      2818144.       13961      195.2487      216.9626         413

maxsma 89.54305

/home/chetna/Documents/ap_phot_wise/w4/w4_0258+35.fits

msky =  201.7408

stdev =  0.02208086

new sma =  81.40277

1sigma aperture params: w4_0258+35 195.2487 216.9626 0.3293196 65.49702

Target values:

bkgd_sub_flux=  1640.6912 total_valid_pixels=  13961.0 mag=  4.962433



w4_1037+30

x0 =  218.8

y0 =  218.6

Masking done

Running ellipse first stage...

55.59918       185.218     0.0033818     0.0473452     0.6390382         INDEF      40.53199         INDEF  -5.927962E-4   8.641471E-4      2.730657      7.330792   1.991753E-5      648623.7        3501      222.6545      208.5474         196

maxsma 55.59918

/home/chetna/Documents/ap_phot_wise/w4/w4_1037+30.fits

msky =  185.2123

stdev =  0.01574039

new sma =  45.94973

1sigma aperture params: w4_1037+30 222.6545 208.5474 0.6390382 40.53199

Target values:

bkgd_sub_flux=  184.3169 total_valid_pixels=  2397.0 mag=  7.336087



w4_1221-423

x0 =  219.3

y0 =  223.8

Masking done

Running ellipse first stage...

61.1591      126.2139   0.001477606    0.02630803      0.305533         INDEF     -56.75602         INDEF  -6.729335E-4         INDEF      2.796503      7.747232   1.249720E-5      1030505.        8161      206.6325      217.9682         317


55.59918      126.2186   0.002183748    0.03705944      0.305533         INDEF     -56.75602         INDEF  -8.411668E-4   4.742298E-4      2.730657      7.747191   1.883489E-5      851783.2        6745      206.6325      217.9682         288

maxsma 45.94973

manually entered stats: msky= 126.238 stdev= 0.028

new sma =  21.43589

1sigma aperture params: w4_1221-423 219.2685 222.8654 0.09681354 -24.31677

Target values:

bkgd_sub_flux=  311.886 total_valid_pixels=  1303.0 mag=  6.76501



In [17]:
#constructing model & residual images

iraf.isophote.bmodel.setParam('table', input_dir+target_name+'_isofit.tab')
iraf.isophote.bmodel.setParam('output', input_dir+ target_name+'_model.fits')
iraf.isophote.bmodel.setParam('parent',input_dir+target_name+'.fits')
iraf.isophote.bmodel.setParam('interp','spline')
iraf.isophote.bmodel(mode='h')

iraf.images.imutil.imarith.setParam('operand1', input_dir+target_name+'.fits')
iraf.images.imutil.imarith.setParam('op', '-')
iraf.images.imutil.imarith.setParam('operand2', input_dir+target_name+'_model.fits')
iraf.images.imutil.imarith.setParam('result', input_dir+target_name+'_residual.fits')
iraf.images.imutil.imarith(mode='h')

#list all files in a directory
import os
files = [ f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir,f)) ]
for f in files:
    if f.endswith('.fits'):
        print f



w4_0258_model.fits
w4_1221-423_model.fits
w4_0258_residual.fits
w4_1037+30.fits
w4_1037_model.fits
w4_1221-423_residual.fits
w4_1037_residual.fits
w4_1221-423.fits
w4_0258+35.fits
